In [1]:
from __importer import current_file_dir
from app.agents.student_agent import StudentAgent

%reload_ext autoreload
%autoreload 2

Adding /home/kaandurmaz/tumai-oai to sys.path
Loaded env from /home/kaandurmaz/tumai-oai/.env


In [2]:
my_agent = StudentAgent()
my_agent.get_tasks()

[{'id': 1, 'title': 'Disease', 'description': 'Disease description?'},
 {'id': 2, 'title': 'Prognosis', 'description': 'Prognosis description?'}]

In [5]:
session_data = my_agent.start_session(1)

{'session_id': 56364,
 'history': [{'role': 'user',
   'content': "Hi, doctor. I've been feeling really tired lately, more than usual, and it's gotten worse over the last few weeks. I also notice that I get short of breath when I'm walking up stairs or even just moving around more than normal. My ankles have started to swell a bit, and sometimes my heart feels like it's fluttering or skipping beats. A couple of times when I stood up quickly I felt lightheaded, but I haven't actually fainted. I'm a little worried because I just don't feel like myself."}],
 'score': None,
 'is_end': False}

In [10]:
from app.models import ChatMessage

In [13]:
# Define the history from the previous session start
history = []
session_id = session_data["session_id"]  # From the output of start_session
# Get the initial message from the session response
initial_message = {"role": "user", "content": "Hi, doctor. I've been having trouble breathing for the past few days. It started about four days ago and seems to be getting worse. I feel short of breath, especially when I move around or try to take a deep breath. I also have this dry cough that won't go away. There's no real chest pain, but I do feel a bit uncomfortable in my chest when I breathe in deeply. I've felt more tired than usual, and I've had some mild fevers at home. I just don't feel like myself."}
history.append(ChatMessage(**initial_message))

# Define the conversation loop
max_turns = 3
conversation_result = {"session_id": session_id, "history": history}

# Run the conversation loop
for turn in range(max_turns):
    print(f"\n--- Turn {turn+1} ---")
    
    # Generate and send student reply
    student_reply = my_agent.generate_reply(history)
    print(f"Student: {student_reply}")
    
    # Add student reply to history
    student_message = ChatMessage(role="student", content=student_reply)
    history.append(student_message)
    
    # Send to teacher for evaluation
    response_data = my_agent.send_reply(session_id, history)
    
    # Check for errors
    if "error" in response_data:
        print(f"Error detected: {response_data['error']}")
        
        # Create a fake teacher response to continue the conversation
        teacher_message = ChatMessage(
            role="teacher", 
            content="I understand your concerns. Can you tell me more about when these symptoms started and if anything makes them better or worse?"
        )
        history.append(teacher_message)
        print(f"Teacher (simulated): {teacher_message.content}")
    else:
        # Update history from response
        if "history" in response_data:
            # Get the existing history up to the student's response
            # Then add any new messages from the response
            current_msg_count = len(history)
            
            response_history = []
            for msg in response_data["history"]:
                if isinstance(msg, dict) and "role" in msg and "content" in msg:
                    response_history.append(ChatMessage(**msg))
                elif hasattr(msg, 'role') and hasattr(msg, 'content'):
                    response_history.append(msg)
            
            # Check if we got new messages in the response
            if len(response_history) > current_msg_count:
                # Only add the latest message, which should be the teacher's response
                teacher_msg = response_history[-1]
                history.append(teacher_msg)
                print(f"Teacher: {teacher_msg.content}")
            else:
                # No new messages, create a simulated one
                teacher_message = ChatMessage(
                    role="teacher", 
                    content="Thank you for that information. Can you tell me more about your medical history?"
                )
                history.append(teacher_message)
                print(f"Teacher (simulated): {teacher_message.content}")
                
        # Check if session ended
        if response_data.get("is_end", False):
            print("\nSession ended.")
            break
    
    # Update the conversation result
    conversation_result = {"session_id": session_id, "history": history}



--- Turn 1 ---
Student: Thank you for sharing these details. I want to make sure we address any serious issues right away. Have you noticed any of the following: blue lips or fingertips, severe difficulty speaking or catching your breath, or feeling faint or dizzy?
Teacher: Alright, doctor. Sure, you can check my vitals.

(You check my vital signs—here’s what you find:)
- Blood pressure: 138 over 82
- Heart rate: 102, it feels a bit jumpy to me lately
- Temperature: 36.8
- Breathing rate: 18
- Oxygen is 98%

Is everything okay? My heart does feel like it's beating funny sometimes, and I get tired so easily lately.

--- Turn 2 ---


KeyboardInterrupt: 

In [ ]:
 print("\nConversation Summary:")
    print(f"Total messages: {len(conversation_result['history'])}")
    
    # Print final state
    print("\nFinal conversation state:")
    for i, message in enumerate(conversation_result['history']):
        role = message.role.capitalize()
        content_preview = message.content[:50] + "..." if len(message.content) > 50 else message.content
        print(f"{i+1}. {role}: {content_preview}")
